In [1]:
# required Python imports
import numpy as np
import pandas as pd
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.3.07 imported


In [2]:
# STEP 1.1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=27, lower_bounds=(0, 9, 0)))

# STEP 1.2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

# STEP 1.3: Generate 16 measures at different azimuth angles
angles = np.arange(0, 360, 45/2)

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

    # get the indexes of specific angles
    indexes = f.find_nearest(xy_vectors['angles'], angles)

    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals)

In [3]:
estimates = []

for index, orientation in enumerate(database.euler):
    print('Real:', np.around(orientation, 0))

    measures = np.column_stack((database.T_values[index],
                                database.azimuths[index],
                                np.full_like(database.azimuths[index], 90)))
    
    result = f.find_orientation(measurements=measures, params=(90, 50, 20), num_guesses=20)

    # store estimates
    estimates.append(np.around(result.x, 0))
    print('----------------------')

Real: [0 9 0]
Calculated orientation: [  1.   9. 179.]
----------------------
Real: [ 0  9 27]
Calculated orientation: [ 1.  9. 26.]
----------------------
Real: [ 0  9 54]
Calculated orientation: [ 1.  9. 53.]
----------------------
Real: [ 0  9 81]
Calculated orientation: [ 1.  9. 80.]
----------------------
Real: [  0   9 108]
Calculated orientation: [  1.   9. 107.]
----------------------
Real: [  0   9 135]
Calculated orientation: [  1.   9. 134.]
----------------------
Real: [  0   9 162]
Calculated orientation: [  1.   9. 161.]
----------------------
Real: [ 0 36  0]
Calculated orientation: [  1.  36. 180.]
----------------------
Real: [ 0 36 27]
Calculated orientation: [ 1. 36. 27.]
----------------------
Real: [ 0 36 54]
Calculated orientation: [ 1. 36. 54.]
----------------------
Real: [ 0 36 81]
Calculated orientation: [ 1. 36. 81.]
----------------------
Real: [  0  36 108]
Calculated orientation: [  1.  36. 108.]
----------------------
Real: [  0  36 135]
Calculated orient

In [4]:
# creates a list with 112 values with repeated numbers every 7 values
column_x = []
for i in range(1, 17):
    for j in range(7):
        column_x.append(i)

In [5]:
data2 = pd.DataFrame(estimates, columns = ['phi1', 'PHI', 'phi2'])
data2['X'] = column_x
data2['Y'] = 1
# data2['phase'] = 'olivine'

data2

,phi1,PHI,phi2,X,Y
0,1.0,9.0,179.0,1,1
1,1.0,9.0,26.0,1,1
2,1.0,9.0,53.0,1,1
3,1.0,9.0,80.0,1,1
4,1.0,9.0,107.0,1,1
...,...,...,...,...,...
107,81.0,90.0,54.0,16,1
108,81.0,90.0,81.0,16,1
109,81.0,90.0,108.0,16,1
110,81.0,90.0,135.0,16,1


In [6]:
data1 = pd.DataFrame(database.euler, columns = ['phi1', 'PHI', 'phi2'])
data1['X'] = column_x
data1['Y'] = 1
# data1['phase'] = 'olivine'

data1

,phi1,PHI,phi2,X,Y
0,0,9,0,1,1
1,0,9,27,1,1
2,0,9,54,1,1
3,0,9,81,1,1
4,0,9,108,1,1
...,...,...,...,...,...
107,81,90,54,16,1
108,81,90,81,16,1
109,81,90,108,16,1
110,81,90,135,16,1


In [7]:
data1.to_csv('real_orientations.csv', index=False)
data2.to_csv('estimated_orientations.csv', index=False)

Result from MTEX. See ``plot.ori.m`` file for details.

![pole_figures](https://raw.githubusercontent.com/bertopadron/FTIR-EBSD/main/docs/pole_fig_comparison.jpg)

Only one orientation fails (using the default algorithm with 20 guesses). To follow up!